### **Langfun for Langfun: Personalized Langfun Tutorials**

##### **Motivation**
<a name="Motivation"></a>

Imagine learning a new programming language where the compiler not only processes your code but also adapts to become your personal tutor. That's exactly what we're achieving with "Langfun for Langfun." By leveraging Langfun's powerful language manipulation capabilities, we've created a agent that generates customized tutorials that evolve based on your background, style, pace, and specific interests.


<hr>        

In [19]:
## import langfun and related packages
import langfun as lf
import pyglove as pg
from langfun.core.structured import function_generation
from langfun.core.modalities import pdf as pdf_lib
from langfun.core.template import Template
from langfun.core.template import TemplateRenderEvent
import pandas as pd
import json
from typing import Literal, Annotated
from datetime import datetime
import json
from IPython.display import display, JSON, HTML, Image


## optional, this is where setup the API_KEY
from dotenv import load_dotenv
## load the keys
load_dotenv(dotenv_path='../../.env')  # This loads your .env file
import os

youtube_key = os.environ.get('YOUTUBE_KEY')
claude_key = os.environ.get('CLAUDE_API_KEY')
gemini_key = os.environ.get('GEMINI_API_KEY')
openai_key = os.environ.get('OPENAI_API_KEY')

## build the LLM crew
lm_claude = lf.llms.Claude35Sonnet(api_key=claude_key, temperature=0.0)
lm_openai = lf.llms.Gpt4o(api_key=openai_key, temperature=0.6)
lm_gemini = lf.llms.GeminiExp_20241114(api_key=gemini_key, temperature=0.6)


In [15]:
import PyPDF2
from io import BytesIO

with open('./_static/langfun101.pdf', 'rb') as f:
    pdf_bytes = f.read()
    
# Create a PDF reader object
pdf_reader = PyPDF2.PdfReader(BytesIO(pdf_bytes))

pages_content = []
for page in pdf_reader.pages:
    pages_content.append(page.extract_text())


In [17]:
pages_content[0]

"Langfun 101: Getting Started with Langfun\nOpen in Colab Open in Colab\nEffective programming of Large Language Models (LLMs) demands a seamless\nintegration of natural language text with structured data. Langfun, leveraging PyGlove's\nsymbolic objects, provides a simple yet powerful interface for mapping between Python\nobjects with the assistance of LLMs. The input/output objects may include natural\nlanguage texts (in string form), structured data (objects of a specific class), modalities\n(such as images), and more. The unified API for accomplishing all conceivable mappings\nis lf.query.\n!pip install langfun\nimport langfun as lf\nimport pyglove as pg\n# Set environment variable for OPENAI_KEY so it could be used throughout all\nimport os\nos.environ['OPENAI_API_KEY' ] = '<replace-with-your-open-ai-key>'\nNatural Language -> Natural Language\nA simple example\nr = lf.query('Who is Larry Page' , lm=lf.llms.Gpt4o())\nprint(r)\nIn\xa0[\xa0]:\nIn\xa0[\xa0]:\nIn\xa0[\xa0]:\nIn\xa0[\xa

In [18]:
QA_assistant = lf.llms.Gpt4o(api_key=openai_key, temperature=0.6)
code_assistant = lf.llms.Claude35Sonnet(api_key=claude_key, temperature=0.0)

In [50]:
class QAItem(pg.Object):
    question: str
    answers: str
    code_examples: Annotated[lf.PythonCode,  "Python code for example cases"]

class QAResult(pg.Object):
    item_pair: list[QAItem]

In [51]:
Q_A_Prelimitary = lf.query(prompt="""I am introducing a Langfun package,  it's a tool that seamlessly integrates with LLM,  
                           object oriented prompting.  super powerful, and user could easily leverage it to build agent, 
                           and switch different LLMs to pursue best performances within agents.   
                           Help me come up with a structure of QA lists that explain to user intuitively with code examples. 
                           Please considering the users have a standard framework, input, output when you answer the questions.  Please grounded on content:{{content}}""", 
         content=pages_content, 
         lm=code_assistant, 
         schema=QAResult)

In [52]:
Q_A_Prelimitary

0item_pair[0],"QAItem(...)QAItem( question='What is Langfun and what problem does it solve?', answers='Langfun is a Python package that provides seamless integration between natural language and structured data when working with LLMs. It solves the challenge of mapping between Python objects and LLM outputs by providing a unified API (lf.query) that can han...', code_examples=PythonCode( source=""\nimport langfun as lf\nimport pyglove as pg\n\n# Simple text to text query\nr = lf.query('Who is Larry Page', lm=lf.llms.Gpt4o())\n\n# Text to structured data\nclass Answer(pg.Object):\n result: int\n \nr = lf.query('The result of one plus two is three', schema=An..."" ) )questionitem_pair[0].questionstr'What is Langfun and what problem does it solve?''What is Langfun and what problem does it solve?'answersitem_pair[0].answersstr'Langfun is a Python package that provides seamless integration between natural language and structured data when working with LLMs. It solves the challenge of mapping between Python objects and LLM outputs by providing a unified API (lf.query) that can han...'Langfun is a Python package that provides seamless integration between natural language and structured data when working with LLMs. It solves the challenge of mapping between Python objects and LLM outputs by providing a unified API (lf.query) that can handle various types of mappings including text-to-text, text-to-structured data, and structured data-to-structured data.code_examplesitem_pair[0].code_examplesPythonCode(...)PythonCode( source=""\nimport langfun as lf\nimport pyglove as pg\n\n# Simple text to text query\nr = lf.query('Who is Larry Page', lm=lf.llms.Gpt4o())\n\n# Text to structured data\nclass Answer(pg.Object):\n result: int\n \nr = lf.query('The result of one plus two is three', schema=An..."" )sourceitem_pair[0].code_examples.sourcestr""\nimport langfun as lf\nimport pyglove as pg\n\n# Simple text to text query\nr = lf.query('Who is Larry Page', lm=lf.llms.Gpt4o())\n\n# Text to structured data\nclass Answer(pg.Object):\n result: int\n \nr = lf.query('The result of one plus two is three', schema=An..."" import langfun as lf import pyglove as pg # Simple text to text query r = lf.query('Who is Larry Page', lm=lf.llms.Gpt4o()) # Text to structured data class Answer(pg.Object): result: int r = lf.query('The result of one plus two is three', schema=Answer, lm=lf.llms.Gpt4o())"
1item_pair[1],"QAItem(...)QAItem( question='How does Langfun handle structured data parsing and generation?', answers=""Langfun uses PyGlove's symbolic objects to define output schemas as Python classes. When provided with a schema, LLMs can parse natural language into structured objects or directly generate structured outputs. It supports complex structures with inheritanc..."", code_examples=PythonCode( source=""\n# Define schema for parsing\nclass Country(pg.Object):\n name: str\n population: int\n capital: str\n\n# Parse text into structured data\nr = lf.query(\n 'The US has population of 330M with capital Washington DC',\n schema=Country,\n lm=lf.llms.Gpt4o()\n..."" ) )questionitem_pair[1].questionstr'How does Langfun handle structured data parsing and generation?''How does Langfun handle structured data parsing and generation?'answersitem_pair[1].answersstr""Langfun uses PyGlove's symbolic objects to define output schemas as Python classes. When provided with a schema, LLMs can parse natural language into structured objects or directly generate structured outputs. It supports complex structures with inheritanc...""Langfun uses PyGlove's symbolic objects to define output schemas as Python classes. When provided with a schema, LLMs can parse natural language into structured objects or directly generate structured outputs. It supports complex structures with inheritance, composition and can handle multimodal inputs like images.code_examplesitem_pair[1].code_examplesPythonCode(...)PythonCode( source=""\n# Define schema for parsing\nclass Country(pg.Object):\n name: 